In [1]:
import pandas as pd

file_path = '/Users/johannessolibieda/Documents/GitHub/Its-Wekk/4 - Data/4 - Johannes/Target_Variable_St.Johannsplatz_cleaned_hourly.csv'
data = pd.read_csv(file_path, delimiter=';')

# Display the first few rows of the dataframe
data = data.sort_values(by='Datum').reset_index(drop=True)
data.head()

,Datum,PM10 (Stundenmittelwerte),PM2.5 (Stundenmittelwerte)
0,2022-01-01 00:00:00,83.840,71.870
1,2022-01-01 01:00:00,91.973,78.909
2,2022-01-01 02:00:00,68.396,58.632
3,2022-01-01 03:00:00,49.625,42.563
4,2022-01-01 04:00:00,41.854,35.799


In [2]:
missing_values = data.isnull().sum()
print(missing_values)

Datum                           0
PM10 (Stundenmittelwerte)     263
PM2.5 (Stundenmittelwerte)     23
dtype: int64


In [3]:
# Convert 'Datum' column to datetime
data['Datum'] = pd.to_datetime(data['Datum'])

# Extract hour and weekday from 'Datum' column
data['Hour'] = data['Datum'].dt.hour
data['Weekday'] = data['Datum'].dt.weekday

# Function to fill missing values with mean of the same hour on the same weekday
def fill_missing_values(df, column):
    return df[column].fillna(df.groupby(['Hour', 'Weekday'])[column].transform('mean'))

# Fill missing values for PM10 and PM2.5 columns
data['PM10 (Stundenmittelwerte)'] = fill_missing_values(data, 'PM10 (Stundenmittelwerte)')
data['PM2.5 (Stundenmittelwerte)'] = fill_missing_values(data, 'PM2.5 (Stundenmittelwerte)')

# Drop the temporary columns
data.drop(columns=['Hour', 'Weekday'], inplace=True)

# Display the first few rows of the dataframe to verify
data.head()

,Datum,PM10 (Stundenmittelwerte),PM2.5 (Stundenmittelwerte)
0,2022-01-01 00:00:00,83.840,71.870
1,2022-01-01 01:00:00,91.973,78.909
2,2022-01-01 02:00:00,68.396,58.632
3,2022-01-01 03:00:00,49.625,42.563
4,2022-01-01 04:00:00,41.854,35.799


In [5]:
missing_values_after_fill = data.isnull().sum()
print(missing_values_after_fill)

Datum                         0
PM10 (Stundenmittelwerte)     0
PM2.5 (Stundenmittelwerte)    0
dtype: int64


In [7]:
# Create a complete date range from the minimum to the maximum date in the data
complete_date_range = pd.date_range(start=data['Datum'].min(), end=data['Datum'].max(), freq='H')

# Find the missing timestamps by comparing the complete date range with the 'Datum' column
missing_timestamps = complete_date_range.difference(data['Datum'])

print("Missing timestamps:")
print(missing_timestamps)
# Count the number of missing hours
missing_hours_count = len(missing_timestamps)
print(f"Number of missing hours: {missing_hours_count}")

Missing timestamps:
DatetimeIndex(['2022-02-17 15:00:00', '2022-02-17 16:00:00',
               '2022-02-17 17:00:00', '2022-03-27 01:00:00',
               '2022-03-27 02:00:00', '2022-08-23 15:00:00',
               '2022-08-23 16:00:00', '2022-08-23 17:00:00',
               '2022-10-30 01:00:00', '2023-03-26 01:00:00',
               '2023-03-26 02:00:00', '2023-04-26 09:00:00',
               '2023-10-29 01:00:00', '2024-01-19 15:00:00',
               '2024-03-31 01:00:00', '2024-03-31 02:00:00',
               '2024-05-07 12:00:00', '2024-07-07 21:00:00',
               '2024-07-07 22:00:00', '2024-07-07 23:00:00',
               '2024-07-08 00:00:00', '2024-07-08 01:00:00',
               '2024-07-08 02:00:00', '2024-07-08 03:00:00',
               '2024-07-08 04:00:00', '2024-07-08 05:00:00',
               '2024-07-08 06:00:00', '2024-07-14 03:00:00',
               '2024-07-14 04:00:00', '2024-07-14 05:00:00',
               '2024-10-27 01:00:00'],
              dtype='datet

/var/folders/rt/8pf1f2_x7sj69ytpkljsggxr0000gn/T/ipykernel_53461/337367490.py:2: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  complete_date_range = pd.date_range(start=data['Datum'].min(), end=data['Datum'].max(), freq='H')


In [8]:
# Reindex the dataframe to include the complete date range
data = data.set_index('Datum').reindex(complete_date_range).reset_index()
data.rename(columns={'index': 'Datum'}, inplace=True)

# Interpolate the missing values
data['PM10 (Stundenmittelwerte)'] = data['PM10 (Stundenmittelwerte)'].interpolate()
data['PM2.5 (Stundenmittelwerte)'] = data['PM2.5 (Stundenmittelwerte)'].interpolate()

# Display the first few rows of the dataframe to verify
data.head()

,Datum,PM10 (Stundenmittelwerte),PM2.5 (Stundenmittelwerte)
0,2022-01-01 00:00:00,83.840,71.870
1,2022-01-01 01:00:00,91.973,78.909
2,2022-01-01 02:00:00,68.396,58.632
3,2022-01-01 03:00:00,49.625,42.563
4,2022-01-01 04:00:00,41.854,35.799


In [9]:
# Create a complete date range from the minimum to the maximum date in the data
complete_date_range = pd.date_range(start=data['Datum'].min(), end=data['Datum'].max(), freq='H')

# Find the missing timestamps by comparing the complete date range with the 'Datum' column
missing_timestamps = complete_date_range.difference(data['Datum'])

print("Missing timestamps:")
print(missing_timestamps)

# Count the number of missing hours
missing_hours_count = len(missing_timestamps)
print(f"Number of missing hours: {missing_hours_count}")

Missing timestamps:
DatetimeIndex([], dtype='datetime64[ns]', freq='h')
Number of missing hours: 0


/var/folders/rt/8pf1f2_x7sj69ytpkljsggxr0000gn/T/ipykernel_53461/2558750331.py:2: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  complete_date_range = pd.date_range(start=data['Datum'].min(), end=data['Datum'].max(), freq='H')


In [10]:
data.to_csv('10Targcleaned.csv', index=False)